## Select runs based on scaling restrictions and assumptions in AH2010

In this notebook I select the runs that match the dynamical restrictions imposed in \\Allen and Hickey, 2010\\ (AH2010). 

These restictions are:

1) Nearly uniform incoming flow along the length of the canyon.

2) Weak to moderate flow: $\mathcal{F}R_w<0.2$, where $\mathcal{F}=R_o/(0.9+R_o)$

3) Uniform stratification over the depth of upwelling.

4) Shallow shelf break $B_s=NH_s/fL<2$

5) Steep walls and stratified water column so that the bottom boundary layer is arrested within the canyon. See notebook 
EkmanLayerArrestTime.ipynb.

7) Narrow canyon: $W<2a$ where $a=NH_s/f$ is the Rossby raadius of deformation.

8) Regular shape of the canyon at the upstream corner.

9) Sloped continental shelf.

Restrictions 5, 8 and 9 are geometric and satisified by all runs. Restriction 1 is also satisfied in all runs since they are forced in the same way. Restriction 3 is staisfied by all runs too since the initial stratification is uniform throughout the water column.

In the following, we select the runs that satisfy restrictions 2, 4 and 7.


In [5]:
import cmocean as cmo
import matplotlib.pyplot as plt
%matplotlib inline
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr
import os
import sys

In [6]:
# Set appearance options seaborn

sns.set_style('white')
sns.set_context('notebook')

In [7]:
lib_path = os.path.abspath('../PythonScripts/Paper1Figures') # Add absolute path to my python scripts
sys.path.append(lib_path)

import canyon_records 
import nocanyon_records

records = canyon_records.main()
recordsNoC = nocanyon_records.main()

### Define constants and non dimensional numbers

In [8]:
# Constants and scales

L = 6400.0 # canyon length
R = 5000.0 # Upstream radius of curvature
g = 9.81 # accel. gravity
Wsb = 13000 # Width at shelf break
Hs = 150.0 # Shelf break depth

# NOTE: The default values of all functions correspond to the base case

L = 6400.0 # canyon length
R = 5000.0 # Upstream radius of curvature
g = 9.81 # accel. gravity
Wsb = 13000 # Width at shelf break
Hs = 147.5 # Shelf break depth
s = 0.005 # shelf slope 
W = 8300 # mid-length width
Hh = 97.5 # head depth
Hr = 132.0 # rim depth at DnS
No = 5.5E-3 # Initial stratification at shelf-break depth
f = 9.66E-5 # Coriois parameter 
kv = 1E-5   # base case background vertical diffusivity
kbg = 1E-5  # base case canyon vertical diffusivity
t = 6.5 # diffusion timescale, days 
U = 0.36 # incoming velocity base case, m/s

# NOTE: The default values of all functions correspond to the base case

def Dh(f=9.66E-4,L=6400.0,N=5.5E-3):
    '''Vertical scale Dh'''
    retun(f*L/(N))
    
#def Z(U=0.36,L=6400.0,N=5.5E-3,R=5000.0):
#    '''Depth of upwelling as scaled in Allen and Hickey 2010'''
#    return((1.4*U*(L**(1/2)))/(Nos*(R**(1/2))))

def Z(f, U, R, L, N):
    '''
    INPUT
    U: float: incomming velocity
    R: float: Radius of curvature shelf-break isobath
    f: float: Coriolis Parameter
    L: float: Canyon length
    N: float: initial stratification
    RETURNS
    Depth of upwelling from Allen and Hickey, 2010.
    '''
    return(((f*U*F(Ro(U,f,R))*L)**(0.5))/N)

def Ro(U=0.36,f=9.66E-4,R=5000.0):
    '''Rossby number using radius of curvature as length scale'''
    return(U/(f*R))

def F(Ro):
    '''Function that estimates the hability of the flow to follow isobaths'''
    return(Ro/(0.9+Ro))

def Rw(U=0.36,f=9.66E-5,Wsb=13000.0): 
    '''Rossby number using width at shelf break as length scale'''
    return(U/(f*Wsb))

def Bu(N=5.5E-3,f=9.66E-5,L=6400.0,Hs=150.0):
    '''Burger number'''
    return(N*Hs/(f*L))

def RossbyRad(N=5.5E-3,Hs=150.0,f=9.66E-4):
    '''1st Rossby radius of deformation'''
    return(N*Hs/f)


### Check restrictions

In [9]:
for run in records:
    
    # restriction 2, moderate flow: F*Rw < 0.2
    run.moderate = F(Ro(run.u_mod,run.f,R))*Rw(run.u_mod,run.f,Wsb)
    
    if run.moderate < 0.2:
        print('\033[1;31;32mrun %s satisfies moderate flow restriction: F*Rw = %f < 0.2' %(run.name,run.moderate))
    else:
        print('\033[1;31;30mrun %s doesn\'t satisfy moderate flow restriction: F*Rw = %f' %(run.name,run.moderate))
    
    # restriction 4, shallow shelf break: Bs<2
    run.Burger = Bu(run.N,run.f,L,Hs)
    
    if run.Burger <= 2:
        print('\033[1;31;32mrun %s satisfies shallow shelf restriction: Bu = %f <= 2' %(run.name,run.Burger))
    else:
        print('\033[1;31;30mrun %s doesn\'t satisfy shallow shelf restriction: Bu = %f' %(run.name,run.Burger))
    
    # restriction 7, narrow canyon: Wsb < 2a
    run.narrow = 2*RossbyRad(run.N,Hs,run.f)
    
    if run.narrow >= Wsb:
        print('\033[1;31;32mrun %s satisfies narrow canyon restriction: 2a = %f >= 13000 m \n' %(run.name,run.narrow))
    else:
        print('\033[1;31;30mrun %s doesn\'t satisfy narrow canyon restriction: 2a = %f < 13000 m \n' %(run.name,run.narrow))
    
    
    # Print Ro, Rl, Rw
    RO = Ro(run.u_mod,run.f,R)
    RL = Ro(run.u_mod,run.f,L)
    RWsb = Ro(run.u_mod,run.f,Wsb)
    print('\033[1;31;32mrun %s Rossby numbers are : Ro=%f, RL=%f, RWsb=%f  \n' %(run.name,RO,RL,RWsb))
    


run CNTDIFF_run38 satisfies moderate flow restriction: F*Rw = 0.129862 < 0.2
run CNTDIFF_run38 satisfies shallow shelf restriction: Bu = 1.312193 <= 2
run CNTDIFF_run38 satisfies narrow canyon restriction: 2a = 16796.066253 >= 13000 m 

run CNTDIFF_run38 Rossby numbers are : Ro=0.745342, RL=0.582298, RWsb=0.286670  

run CNTDIFF_run37 satisfies moderate flow restriction: F*Rw = 0.128190 < 0.2
run CNTDIFF_run37 satisfies shallow shelf restriction: Bu = 1.312193 <= 2
run CNTDIFF_run37 satisfies narrow canyon restriction: 2a = 16796.066253 >= 13000 m 

run CNTDIFF_run37 Rossby numbers are : Ro=0.739130, RL=0.577446, RWsb=0.284281  

run CNTDIFF_run36 satisfies moderate flow restriction: F*Rw = 0.108045 < 0.2
run CNTDIFF_run36 satisfies shallow shelf restriction: Bu = 1.312193 <= 2
run CNTDIFF_run36 satisfies narrow canyon restriction: 2a = 16796.066253 >= 13000 m 

run CNTDIFF_run36 Rossby numbers are : Ro=0.662526, RL=0.517598, RWsb=0.254818  

run CNTDIFF_run45 satisfies moderate flow r

From all the runs, we need to exclude:
    
    * Run 44 and 46: the stratification is too small and they do not behave as narrow canyons since the rossby radius ends up being too big.
    
    * Run 52 does not satisfy the moderate flow restriction because f is too small, nor the shallow shelf restriction.
    
    * run 71 does not satisfy the moderate flow restriction because f is too small, nor the shallow shelf restriction. Still , it is close to the bounds, so I'll keep it.
    
    * Same with run 51, it does not satisfy the moderate flow restriction but it is close to the bounds.

## Made-up runs

In [18]:
uu = 0.13
NN = 0.005
ff = 9.66E-5


# restriction 2, moderate flow: F*Rw < 0.2
moderate = F(Ro(uu,ff,R))*Rw(uu,ff,Wsb)
    
if moderate < 0.2:
    print('\033[1;31;32mrun satisfies moderate flow restriction: F*Rw = %f < 0.2' %(moderate))
else:
    print('\033[1;31;30mrun doesn\'t satisfy moderate flow restriction: F*Rw = %f' %(moderate))
    
# restriction 4, shallow shelf break: Bs<2
Burger = Bu(NN,ff,L,Hs)
    
if Burger <= 2:
    print('\033[1;31;32mrun satisfies shallow shelf restriction: Bu = %f <= 2' %(Burger))
else:
    print('\033[1;31;30mrun doesn\'t satisfy shallow shelf restriction: Bu = %f' %(Burger))
    
# restriction 7, narrow canyon: Wsb < 2a
Narrow = 2*RossbyRad(NN,Hs,ff)
    
if Narrow >= Wsb:
    print('\033[1;31;32mrun satisfies narrow canyon restriction: 2a = %f >= 13000 m \n' %(Narrow))
else:
    print('\033[1;31;30mrun doesn\'t satisfy narrow canyon restriction: 2a = %f < 13000 m \n' %(Narrow))
    
    
# Print Ro, Rl, Rw
RO = Ro(uu,ff,R)
RL = Ro(uu,ff,L)
RWsb = Ro(uu,ff,Wsb)
print('\033[1;31;32mrun Rossby numbers are : Ro=%f, RL=%f, RWsb=%f  \n' %(RO,RL,RWsb))
    


run satisfies moderate flow restriction: F*Rw = 0.023831 < 0.2
run satisfies shallow shelf restriction: Bu = 1.192902 <= 2
run satisfies narrow canyon restriction: 2a = 15269.151139 >= 13000 m 

run Rossby numbers are : Ro=0.269151, RL=0.210274, RWsb=0.103520  

